In [7]:
import requests
from bs4 import BeautifulSoup
import re
import urllib3

# Disable SSL warnings for this session
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Base URL
base_url = 'https://laws.boe.gov.sa'

# Page to fetch
folder_url = base_url + '/BoeLaws/Laws/Folders/1'

# Get the HTML content (disable SSL verification)
response = requests.get(folder_url, verify=False)
response.encoding = 'utf-8'  # Ensure Arabic renders correctly

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    # Regex pattern for internal law detail links
    pattern = re.compile(r"^/BoeLaws/Laws/LawDetails/[\w-]+/\d+$")

    # Extract and deduplicate links
    links = set()
    for a in soup.find_all('a', href=True):
        href = a['href']
        if pattern.match(href):
            links.add(base_url + href)

    # Save to file
    with open('collected links.txt', 'w', encoding='utf-8') as f:
        for link in sorted(links):
            f.write(link + '\n')

    print(f"✅ Fetched and saved {len(links)} links to 'collected links.txt'")
else:
    print(f"❌ Failed to fetch the page. Status code: {response.status_code}")

✅ Fetched and saved 517 links to 'collected links.txt'


In [6]:
import os
import re
import requests
from bs4 import BeautifulSoup
from docx import Document
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
import urllib3

# Disable SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Output folder
output_dir = 'laws_output'
os.makedirs(output_dir, exist_ok=True)

# Clean illegal characters from filenames
def sanitize_filename(name):
    name = re.sub(r'[\\/*?:"<>|]', "", name)
    return name.strip().replace('\n', ' ')[:100]

# Apply RTL direction to a paragraph
def set_rtl(paragraph):
    p = paragraph._p
    pPr = p.get_or_add_pPr()
    bidi = OxmlElement('w:bidi')
    bidi.set(qn('w:val'), '1')
    pPr.append(bidi)

# Read links
with open('collected links.txt', 'r', encoding='utf-8') as f:
    links = [line.strip() for line in f if line.strip()]

# Track successful saves
success_count = 0

# Process each link
for idx, url in enumerate(links, 1):
    try:
        response = requests.get(url, verify=False)  # Disable SSL verification
        response.encoding = 'utf-8'

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            # Get title for filename
            h3 = soup.find('h3', class_='center')
            title = h3.get_text(strip=True) if h3 else f'law_{idx:03}'
            filename = sanitize_filename(title) + '.docx'

            # Get law text
            law_div = soup.find('div', id='divLawText')
            law_text = law_div.get_text(separator='\n', strip=True) if law_div else ''

            if law_text:
                doc = Document()
                para = doc.add_paragraph(law_text)
                set_rtl(para)
                file_path = os.path.join(output_dir, filename)
                doc.save(file_path)
                print(f"✅ Saved: {file_path}")
                success_count += 1
            else:
                print(f"⚠️ No text in divLawText at: {url}")
        else:
            print(f"❌ Failed to fetch {url} — Status: {response.status_code}")
    except Exception as e:
        print(f"❌ Exception for {url}: {e}")

# Summary
print(f"\n🎉 Done! Total laws converted to DOCX: {success_count} / {len(links)}")

✅ Saved: laws_output\تنظيم مؤسسة البريد السعودي.docx
✅ Saved: laws_output\نظام دارة الملك عبد العزيز.docx
✅ Saved: laws_output\تنظيم بنك التصدير والاستيراد السعودي.docx
✅ Saved: laws_output\نظام الجمعيات التعاونية.docx
✅ Saved: laws_output\اللائحة التنظيمية لدور الأحداث.docx
✅ Saved: laws_output\تنظيم المجلس الاقتصادي الأعلى.docx
✅ Saved: laws_output\تنظيم مدينة الملك عبدالعزيز للعلوم والتقنية.docx
✅ Saved: laws_output\نظام العلم للمملكة العربية السعودية.docx
✅ Saved: laws_output\النظام الأساسي للاتحادات الرياضية واللجنة الأولمبية العربية السعودية.docx
✅ Saved: laws_output\نظام الدفاتر التجارية.docx
✅ Saved: laws_output\اللائحة التنفيذية لنظام نزع ملكية العقارات للمنفعة العامة ووضع اليد المؤقت على العقار.docx
✅ Saved: laws_output\الترتيبات التنظيمية للهيئة السعودية لتسويق الاستثمار.docx
✅ Saved: laws_output\نظام مكافحة غسل  الأموال.docx
✅ Saved: laws_output\نظام العمل.docx
✅ Saved: laws_output\نظام التقاعد العسكري.docx
✅ Saved: laws_output\اللائحة التنظيمية لمراكز التنمية الاجتماعية.do